In [1]:
import requests
import threading
import time

In [2]:
def submit_prompt(prompt, time_difference):
    # Endpoint URL
    url = 'http://127.0.0.1:5000/submit'
    
    # Prepare the data payload
    data = {'input': prompt}
    
    start_time = time.time()
    # Send the POST request
    response = requests.post(url, json=data)

    task_id = response.json()['task_id']

    while True:
        update_url = f'http://127.0.0.1:5000/status/llm/{task_id}'
        update_response = requests.get(update_url)
        status = update_response.json()['status']
        if status == 'COMPLETE':
            print(f'Thread for prompt "{prompt}" received completion status: {update_response.json()}')
            time_difference.append(time.time() - start_time)
            break
        time.sleep(1)

In [3]:
# Path to the file containing prompts
file_path = 'questions.txt'

# List to hold all threads
threads = []

time_differences = []

# Read prompts from the file
with open(file_path, 'r') as file:
    prompts = file.readlines()

prompts = prompts[:30]

# Creating and starting threads
for prompt in prompts:
    # Remove any trailing newline characters from each prompt
    prompt = prompt.strip()
    thread = threading.Thread(target=submit_prompt, args=(prompt, time_differences,))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

Thread for prompt "What was the purpose of the Berlin Wall and how did it affect the connections between different sectors of West Berlin?" received completion status: {'electricity': 16, 'result': "The Berlin Wall was erected by the German Democratic Republic (East Germany) on August 13, 1961, to prevent its citizens from fleeing to the Federal Republic of Germany (West Germany). The wall effectively sealed off East Berlin, which was under the control of the Soviet Union and its satellite state East Germany, from West Berlin, which was under the control of the Western Allies.\n\nThe primary reason for building the wall was to stop the mass exodus of East Germans to West Germany. At the time, East Germany had a struggling economy and many of its citizens were dissatisfied with their living conditions. They saw West Germany as offering greater economic opportunities and personal freedoms, leading to a significant brain drain. In just the first five years after World War II, an estimated

In [6]:
prompts_per_minute = len(prompts) / max(time_differences) * 60
print(f'Prompts per second: {prompts_per_minute}')

Prompts per second: 3.594439934710159
